In [123]:
import xlrd
import itertools
import numpy as np
import xlsxwriter


########################################################################################################

# There is a repetition of a batch in the second test case so I have removed that now it has 49 groups 

#########################################################################################################

# Storing file in the form of dictionary of dictionaries so that batch becomes key and related 
# information of a batch becomes value further containing key which is course and its value 
# further contains multiple key value pairs where key points to the teacher and value to the duration
# of lecture

# Example of a record

# {u'BatchCS16_C': {u'C3': {u'T3': [1.0]}, u'C2': {u'T2': [1.5]}, u'C1': {u'T1': [1.0]},
# u'C6': {u'T6': [1.0]}, u'C5': {u'T5': [1.0]},u'C4': {u'T4': [1.5]}}

# Also storing total batches, courses, classes, teachers for future use

path = './AI_testCase.xlsx'
name = []
workbook = xlrd.open_workbook(path)
worksheet = workbook.sheet_by_index(2)
print 'Classes : ',worksheet.cell_value(1, 1)
print 'Teachers : ',worksheet.cell_value(2, 1)
print 'Rooms : ',worksheet.cell_value(3, 1)
print 'Groups : ',worksheet.cell_value(4, 1)
classes = worksheet.cell_value(1, 1)
teachers = worksheet.cell_value(2, 1)
rooms = worksheet.cell_value(3, 1)
groups = worksheet.cell_value(4, 1)
batch_section_teacher_time_data = {}
for i in range(int(groups)):
    batch_section_teacher_time_data[worksheet.cell_value(7+(i*2), 0)]={}
    name.append(worksheet.cell_value(7+(i*2), 0))
    for j in range(1,7):
        if worksheet.cell_value(7+(i*2), j)!='':
            batch_section_teacher_time_data[worksheet.cell_value(7+(i*2), 0)][worksheet.cell_value(6, j)]={}
            batch_section_teacher_time_data[worksheet.cell_value(7+(i*2), 0)][worksheet.cell_value(6, j)][worksheet.cell_value(7+(i*2), j)] = [worksheet.cell_value(7+(i*2)+1, j)]

Classes :  900.0
Teachers :  100.0
Rooms :  30.0
Groups :  49.0


In [124]:
# Storing teacher preferences from end of file so that teacher constraint of having lectures at specified time 
# could be handled
teacher_preference = {}
stop = 0
i = 0
while stop == 0:
    teacher_preference[worksheet.cell_value(7+(int(groups-1)*2)+3+(i+1), 0)] = worksheet.cell_value(7+(int(groups-1)*2)+3+(i+1), 1)
    if 7+(int(groups-1)*2)+3+(i+2) < worksheet.nrows:
        i = i+1
    else:
        stop = 1

In [125]:
# Storing free days for each batch so that the constraint of batches having specific day off could be catered
# Free days per batch are assigned randomly
free_days = {}
for key, value in sorted(batch_section_teacher_time_data.iteritems()):
    d = np.random.randint(5)
    free_days[key] = d

In [126]:
# Chromosome class containing group, course, teacher, duration, representation chosen for a single chromosome
class Chromosome:
    def __init__(self, g,c,t,d):
        self.group = g
        self.course = c
        self.teacher = t
        self.duration = d

# function provides as many solutions as asked through an argument    
# It basically consists of a loop which runs as many times as the number of solutions required
# In each loop first five lists are initialized each representing a single day of the week
# After that each of those five lists are assigned empty lists equivalent to the number of rooms
# Now comes the loop which runs as many times as the number of classes to be scheduled in a week
# In each loop a random batch is chosen and then a random course is chosen and following that course respective
# teacher and duration is selected and then a chromosome is formed
# Now day is checked and each of the days consisting of 8 hours 9-1 and 1-5
# a day is incremented only if all rooms of thst day are filled and a room is incremented only if 
# the new chromosome doesn't fit in the remaining time of that particular room
# Here care is being taken that is if day is friday so no lecture should be placed in time slot of 1-2
# to cater the constraint of prayer break

def initial_population(elements):
    population = []
    for idx in range(elements):
        d_group = []
        for key, value in sorted(batch_section_teacher_time_data.iteritems()):
            d_group.append(key)        
        monday = []
        for i in range(int(rooms)):
            monday.append([])
        tuesday = []
        for i in range(int(rooms)):
            tuesday.append([]) 
        wednesday = []
        for i in range(int(rooms)):
            wednesday.append([])       
        thursday = []
        for i in range(int(rooms)):
            thursday.append([])    
        friday = []
        for i in range(int(rooms)):
            friday.append([]) 
        tday = 0
        troom = 0
        tduration = 0
        for c in range(int(classes)):
            gn = np.random.randint(int(groups))
            group = d_group[gn]
            coursen = np.random.randint(int(6))
            d_course = []
            for key, value in sorted(batch_section_teacher_time_data[d_group[gn]].iteritems()):
                d_course.append(key)
            course = d_course[coursen]    
            for key, value in batch_section_teacher_time_data[d_group[gn]][d_course[coursen]].iteritems():
                teacher = key
            duration = batch_section_teacher_time_data[d_group[gn]][d_course[coursen]][teacher]
            ch = Chromosome(group,course,teacher,duration[0])
            if tday==0:
                if troom<int(rooms):
                    if tduration<8:
                        monday[troom].append(ch)
                        tduration = tduration + int(ch.duration)
                    else:
                        troom = troom+1
                        tduration = 0
                else:
                    tday = tday+1
                    troom = 0
            elif tday == 1:
                if troom<int(rooms):
                    if tduration<8:
                        tuesday[troom].append(ch)
                        tduration = tduration + int(ch.duration)
                    else:
                        troom = troom+1
                        tduration = 0
                else:
                    tday = tday+1
                    troom = 0
            elif tday == 2:
                if troom<int(rooms):
                    if tduration<8:
                        wednesday[troom].append(ch)
                        tduration = tduration + int(ch.duration)
                    else:
                        troom = troom+1
                        tduration = 0
                else:
                    tday = tday+1
                    troom = 0  
            elif tday == 3:
                if troom<int(rooms):
                    if tduration<8:
                        thursday[troom].append(ch)
                        tduration = tduration + int(ch.duration)
                    else:
                        troom = troom+1
                        tduration = 0
                else:
                    tday = tday+1
                    troom = 0
            elif tday == 4:
                if troom<int(rooms):
                    if tduration<8:
                        if tduration + int(ch.duration)<=4:
                            friday[troom].append(ch)
                            tduration = tduration + int(ch.duration)
                        elif tduration + int(ch.duration)>4 and tduration + int(ch.duration) <= 5:
                            friday[troom].append([])
                            tduration = tduration + (5-tduration)
                            friday[troom].append(ch)
                            tduration = tduration + int(ch.duration)
                        else:
                            friday[troom].append(ch)
                            tduration = tduration + int(ch.duration)
                    else:
                        troom = troom+1
                        tduration = 0
        population.append([monday,tuesday,wednesday,thursday,friday])
    return population    

In [127]:
# functions gives fitness value for each solution given to it through a list
# f_val is a list containing fitness valus of solutions returned at the end of
# the function
# It starts off with a loop which runs equivalent to the number of solutions given
# Now comes another loop which runs equal to the number of days in a week
# In this loop four lists are initialized b_group, b_course, b_teacher, e_group
# b_group contains those batches having multiple classes of same course in a single day to cater the
# constraint that no group can have more than one class of a single course in a single day
# b_course contains those courses for which a batch has multiple classes in a single day
# b_teacher contains lists of teachers assigned to each room so that the constraint that 
# no teacher should have consecutive classes could be catered
# e_group contains lists of groups to which each room is assigned so that the constraint that no group should have
# consecutive classes could be catered
# After initialization of four lists described above, another loop starts which is equivalent to number of rooms
# Here two more lists are initialized, room_teacher, room_group
# room_teacher contains teachers assigned to current room
# room_group contains groups assigned to current room
# After initializations, there is yet another loop which runs equivalent to number of classes scheduled in the
# current room
# Here c_group contains currrent group so that the constraint of providing free days to each group could be catered
# Now current group is checked according to the randomized free days provided in the start and if a group has a 
# lecture on the free day penality is added to the fitness value which is -10 on each violation
# c_teacher contains the current teacher so that the constraint that each teacher having prreferences should have
# lectures at that time could be catered
# After that all preferences are checked and if a teacher having preference violated then penality
# is added to the fitness value
# Now the code just checks that if a teacher or a group has consecutive classes so add penality to 
# the fitness value so that both constraints can be fulfilled


def fitness(p):
    f_val = []
    for a in p:
        fit = 0
        for i in range(5):
            b_group = []
            b_course = []
            b_teacher = []
            e_group = []
            for j in  range(int(rooms)):
                room_teacher = []
                room_group = []
                k = np.shape(a[i][j])[0]
                for l in range(k):
                    c_group = a[i][j][l].group
                    for key, value in free_days.iteritems():
                        if key == c_group:
                            if i != value:
                                fit = fit - 10
                    c_teacher =  a[i][j][l].teacher
                    for key, value in teacher_preference.iteritems() :
                        if key == c_teacher:
                            if value == 'm':
                                if l >3:
                                    fit = fit - 10
                            if value == 'e':
                                if l<4:
                                    fit = fit - 10
                    room_teacher.append(a[i][j][l].teacher)
                    room_group.append(a[i][j][l].group)
                    for m in range(l+1,k):
                        if a[i][j][l].group == a[i][j][m].group:
                            if a[i][j][l].course == a[i][j][m].course:
                                b_group.append(a[i][j][l].group)
                                b_course.append(a[i][j][l].course)                
                b_teacher.append(room_teacher)
                e_group.append(room_group)
            s1 = np.shape(b_teacher)    
            for c_idx in range(int(rooms)):
                if np.shape(s1)[0]>1:
                    for d_idx in range(c_idx,int(rooms)):
                        for e_idx in range(np.shape(b_teacher)[1]):
                            if b_teacher[c_idx][e_idx] == b_teacher[d_idx][e_idx]:
                                fit = fit - 10
                            if e_group[c_idx][e_idx] == e_group[d_idx][e_idx]:
                                fit = fit - 10    
                            if e_idx-1>=0:
                                if b_teacher[c_idx][e_idx] == b_teacher[d_idx][e_idx-1]:
                                    fit = fit - 10
                                if e_group[c_idx][e_idx] == e_group[d_idx][e_idx-1]:
                                    fit = fit - 10      
                            if e_idx+1< np.shape(b_teacher)[1]:
                                if b_teacher[c_idx][e_idx] == b_teacher[d_idx][e_idx+1]:
                                    fit = fit - 10
                                if e_group[c_idx][e_idx] == e_group[d_idx][e_idx+1]:
                                    fit = fit - 10      
            fit = fit - (10*len(b_group))       
        f_val.append(fit)
    return f_val

In [128]:
# It simpy swaps randomly lectures from each room between the both solutions
# the same code is written three times to cater three different scenarios of lectures in a romm
# First is when both have equal
# Second is when first has more
# Third is when  second has more
def crossover(p1,p2):
    for i in range(5):
        for j in  range(int(rooms)):
            if np.shape(p1[i][j])[0]>0 and np.shape(p2[i][j])[0]>0:
                if np.shape(p1[i][j])[0]> np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p2[i][j])[0]):
                        choice = np.random.randint(np.shape(p2[i][j])[0])
                        temp1 = p1[i][j][choice].group
                        temp2 = p1[i][j][choice].course
                        temp3 = p1[i][j][choice].teacher
                        temp4 = p1[i][j][choice].duration
                        p1[i][j][choice].group = p2[i][j][choice].group
                        p1[i][j][choice].course = p2[i][j][choice].course 
                        p1[i][j][choice].teacher = p2[i][j][choice].teacher
                        p1[i][j][choice].duration = p2[i][j][choice].duration
                        p2[i][j][choice].group = temp1
                        p2[i][j][choice].course = temp2
                        p2[i][j][choice].teacher = temp3
                        p2[i][j][choice].duration = temp4
                if np.shape(p1[i][j])[0]< np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p1[i][j])[0]):
                        choice = np.random.randint(np.shape(p1[i][j])[0])
                        temp1 = p1[i][j][choice].group
                        temp2 = p1[i][j][choice].course
                        temp3 = p1[i][j][choice].teacher
                        temp4 = p1[i][j][choice].duration
                        p1[i][j][choice].group = p2[i][j][choice].group
                        p1[i][j][choice].course = p2[i][j][choice].course 
                        p1[i][j][choice].teacher = p2[i][j][choice].teacher
                        p1[i][j][choice].duration = p2[i][j][choice].duration
                        p2[i][j][choice].group = temp1
                        p2[i][j][choice].course = temp2
                        p2[i][j][choice].teacher = temp3
                        p2[i][j][choice].duration = temp4
                if np.shape(p1[i][j])[0] == np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p2[i][j])[0]):
                        choice = np.random.randint(np.shape(p2[i][j])[0])
                        temp1 = p1[i][j][choice].group
                        temp2 = p1[i][j][choice].course
                        temp3 = p1[i][j][choice].teacher
                        temp4 = p1[i][j][choice].duration
                        p1[i][j][choice].group = p2[i][j][choice].group
                        p1[i][j][choice].course = p2[i][j][choice].course 
                        p1[i][j][choice].teacher = p2[i][j][choice].teacher
                        p1[i][j][choice].duration = p2[i][j][choice].duration
                        p2[i][j][choice].group = temp1
                        p2[i][j][choice].course = temp2
                        p2[i][j][choice].teacher = temp3
                        p2[i][j][choice].duration = temp4
    return p1,p2            

In [129]:
# for each lecture it randomly swaps room and day
# Similar to cross over code is written three times so that the same 
# problem could be catered and also some other conditions so that
# code works smoothly such as mutate only if any lectures are scheduled on current day
# and current room

def mutation(p1,p2):
    for i in range(5):
        for j in  range(int(rooms)):
            if np.shape(p1[i][j])[0]>0 and np.shape(p2[i][j])[0]>0:
                if np.shape(p1[i][j])[0]> np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p2[i][j])[0]):
                        choice = np.random.randint(int(rooms))
                        if np.shape(p1[i][j]) == np.shape(p1[i][choice]):
                            temp = p1[i][j][l]
                            p1[i][j][l] = p1[i][choice][l]
                            p1[i][choice][l] = temp
                        choice = np.random.randint(int(rooms))
                        if np.shape(p2[i][j]) == np.shape(p2[i][choice]):
                            temp = p2[i][j][l]
                            p2[i][j][l] = p2[i][choice][l]
                            p2[i][choice][l] = temp
                        choice = np.random.randint(5)
                        shape = np.shape(p1[choice])
                        if len(shape)==2:
                            if np.shape(p1[choice])[1]>0:
                                temp = p1[i][j][l]
                                p1[i][j][l] = p1[choice][j][l]
                                p1[choice][j][l] = temp
                        choice = np.random.randint(5)
                        shape = np.shape(p2[choice])
                        if len(shape)==2:
                            if np.shape(p2[choice])[1]>0:
                                temp = p2[i][j][l]
                                p2[i][j][l] = p2[choice][j][l]
                                p2[choice][j][l] = temp                         
                if np.shape(p1[i][j])[0]< np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p1[i][j])[0]):
                        choice = np.random.randint(int(rooms))
                        if np.shape(p1[i][j]) == np.shape(p1[i][choice]):
                            temp = p1[i][j][l]
                            p1[i][j][l] = p1[i][choice][l]
                            p1[i][choice][l] = temp
                        choice = np.random.randint(int(rooms))
                        if np.shape(p2[i][j]) == np.shape(p2[i][choice]):
                            temp = p2[i][j][l]
                            p2[i][j][l] = p2[i][choice][l]
                            p2[i][choice][l] = temp
                        choice = np.random.randint(5)
                        shape = np.shape(p1[choice])
                        if len(shape)==2:
                            if np.shape(p1[choice])[1]>0:
                                temp = p1[i][j][l]
                                p1[i][j][l] = p1[choice][j][l]
                                p1[choice][j][l] = temp
                        choice = np.random.randint(5)
                        shape = np.shape(p2[choice])
                        if len(shape)==2:
                            if np.shape(p2[choice])[1]>0:
                                temp = p2[i][j][l]
                                p2[i][j][l] = p2[choice][j][l]
                                p2[choice][j][l] = temp
                if np.shape(p1[i][j])[0] == np.shape(p2[i][j])[0]:
                    for l in range(np.shape(p2[i][j])[0]):
                        choice = np.random.randint(int(rooms))
                        if np.shape(p1[i][j]) == np.shape(p1[i][choice]):
                            temp = p1[i][j][l]
                            p1[i][j][l] = p1[i][choice][l]
                            p1[i][choice][l] = temp
                        choice = np.random.randint(int(rooms))
                        if np.shape(p2[i][j]) == np.shape(p2[i][choice]):
                            temp = p2[i][j][l]
                            p2[i][j][l] = p2[i][choice][l]
                            p2[i][choice][l] = temp
                        choice = np.random.randint(5)
                        shape = np.shape(p1[choice])
                        if len(shape)==2:
                            if np.shape(p1[choice])[1]>0:
                                temp = p1[i][j][l]
                                p1[i][j][l] = p1[choice][j][l]
                                p1[choice][j][l] = temp 
                        choice = np.random.randint(5)
                        shape = np.shape(p2[choice])
                        if len(shape)==2:
                            if np.shape(p2[choice])[1]>0:    
                                temp = p2[i][j][l]
                                p2[i][j][l] = p2[choice][j][l]
                                p2[choice][j][l] = temp
    return p1,p2            

In [130]:
#function that runs the whole mechanism of genetic algorithm for time table scheduling
# inputs are population and generations respectively
# for each generation cross over and mutation is carried out among groups of two in 
# a non overlapping manner
#
# Example : if there are 3 solutions then grouping would be 1 and 2
# Example : if there are 4 solutions then grouping would be 1 and 2  3 and 4
#
# after required number of generations solution with maximum fitness is returned
def GA(population,generations):
    p = initial_population(population)
    for gen in range(generations):
        for idx in range(len(p)/2):
            if ((idx)*2)+1<len(p):
                cp1,cp2 = crossover(p[idx*2],p[(idx*2)+1])
                mp1,mp2 = mutation(cp1,cp2)
                del p[idx*2]
                del p[(idx*2)]
                p.append(mp1)
                p.append(mp2)  
    fitness_val = fitness(p)            
    return p[np.argmax(fitness_val)]            

In [131]:
solution = GA(2,10)

In [132]:
# code that writes best solution to a xlsx file in the form of a time table
# room
# day
# group
# course
# teacher
# duration
# A row contains all the classes scheduled on a particular day in a particular room until a blank line
# which indicates end of day
# Colums represent rooms
workbook = xlsxwriter.Workbook('timetable.xlsx')
worksheet = workbook.add_worksheet()
bold = workbook.add_format({'bold': True})
for m in range(5):
    for i in range(int(rooms)):
        worksheet.write(42*m, i+1,i+1,bold)
        worksheet.write(42*m, 0,'Room',bold)
    for j in  range(int(rooms)):
        k = np.shape(solution[m][j])[0] 
        for l in range(k):
            worksheet.set_column((42*m)+(l*5)+1,j+1,15)
            if m==0:
                worksheet.write(((42*m)+l*5)+1, j+1,'Monday',bold)
                worksheet.write(((42*m)+l*5)+1, 0,'Day',bold)
            elif m==1:
                worksheet.write(((42*m)+l*5)+1, j+1,'Tuesday',bold)
                worksheet.write(((42*m)+l*5)+1, 0,'Day',bold)    
            elif m==2:
                worksheet.write(((42*m)+l*5)+1, j+1,'Wednesday',bold)
                worksheet.write(((42*m)+l*5)+1, 0,'Day',bold)   
            elif m==3:
                worksheet.write(((42*m)+l*5)+1, j+1,'Thursday',bold)
                worksheet.write(((42*m)+l*5)+1, 0,'Day',bold)    
            elif m==4:
                worksheet.write(((42*m)+l*5)+1, j+1,'Friday',bold)
                worksheet.write(((42*m)+l*5)+1, 0,'Day',bold)    
            worksheet.write(((42*m)+l*5)+2, j+1,solution[m][j][l].group,bold)
            worksheet.write(((42*m)+l*5)+2, 0,'Group',bold)
            worksheet.write(((42*m)+l*5)+3, j+1,solution[m][j][l].course,bold)
            worksheet.write(((42*m)+l*5)+3, 0,'Course',bold)
            worksheet.write(((42*m)+l*5)+4, j+1,solution[m][j][l].teacher,bold)
            worksheet.write(((42*m)+l*5)+4, 0,'Teacher',bold)
            worksheet.write(((42*m)+l*5)+5, j+1,solution[m][j][l].duration,bold)
            worksheet.write(((42*m)+l*5)+5, 0,'Duration',bold)
workbook.close()